# Escola de Inverno - IESP-UERJ - 2023

## Introdução à Ciência de Dados
Matheus C. Pestana

### Introdução ao Machine Learning e Deep Learning

## Machine Learning

Instalando alguns pacotes necessários

In [ ]:
!pip install huggingface datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.3 MB/s eta 0:00:00


Carregando os pacotes e baixando a base do HuggingFace.

Depois, criando uma variável em que os números da label são o nome das categorias (Negativo, Neutro, Positivo).

In [ ]:
from datasets import load_dataset
import pandas as pd
import re
import numpy as np
import sklearn
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

tqdm.pandas()

tweets1 = load_dataset('cardiffnlp/tweet_sentiment_multilingual', name='portuguese', split='train').to_pandas()
tweets2 = load_dataset('cardiffnlp/tweet_sentiment_multilingual', name='portuguese', split='test').to_pandas()

tweets = pd.concat([tweets1, tweets2])

tweets['label_nome'] = np.select([tweets['label'] == 0, tweets['label'] == 1, tweets['label'] == 2], ['Negativo', 'Neutro', 'Positivo'])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Verificando quantas linhas tem por categoria

In [ ]:
tweets.value_counts('label_nome')

label_nome
Negativo    903
Neutro      903
Positivo    903
dtype: int64

Primeiro, precisamos limpar o texto: deixá-lo todo em minúscula, remover números e stopwords que podem enviesar a análise:

In [ ]:
stopwords = ['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com',
                  'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas',
                  'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está',
                  'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos',
                  'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos',
                  'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem',
                  'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei',
                  'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem',
                  'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvéssemos', 'isso',
                  'isto', 'já', 'lhe', 'lhes', 'mais', 'mas', 'me', 'mesmo', 'meu', 'meus', 'minha', 'minhas', 'muito', 'na', 'não',
                  'nas', 'nem', 'no', 'nos', 'nós', 'nossa', 'nossas', 'nosso', 'nossos', 'num', 'numa', 'o', 'os', 'ou', 'para', 'pela',
                  'pelas', 'pelo', 'pelos', 'por', 'qual', 'quando', 'que', 'quem', 'são', 'se', 'seja', 'sejam', 'sejamos', 'sem', 'ser',
                  'será', 'serão', 'serei', 'seremos', 'seria', 'seriam', 'seríamos', 'seu', 'seus', 'só', 'somos', 'sou', 'sua', 'suas',
                  'também', 'te', 'tem', 'tém', 'temos', 'tenha', 'tenham', 'tenhamos', 'tenho', 'terá', 'terão', 'terei', 'teremos', 'teria',
                  'teriam', 'teríamos', 'teu', 'teus', 'teve', 'tinha', 'tinham', 'tínhamos', 'tive', 'tivemos', 'tiver', 'tivera', 'tiveram',
                  'tivéramos', 'tiverem', 'tivermos', 'tivesse', 'tivessem', 'tivéssemos', 'tu', 'tua', 'tuas', 'um', 'uma', 'você', 'vocês',
                  'vos', 'tá', 'ta', 'pra', 'para', 'sobre', 'todos', 'ainda', 'hoje', 'q', 'de esse', 'aqui', 'hoje', 'dois', 'todo', 'algum',
                  'ser', 'ver', 'dar', 'ir', 'saber', 'falar', 'dever', 'fazer', 'ter', 'achar', 'pedir', 'chegar', 'ficar', 'lá', 'aí', 'então',
                  'né', 'sob', 'desde', 'assim', 'outro', 'tão', 'vez', 'ai', 'tô', 'ah', 'meio', 'caso', 'onde', 'tal', 'quê', 'tudo', 'quanto',
                  'lado', 'agora', 'bem', 'mal', 'nunca', 'ali', 'aaaaaaaa', 'aaaaaaaar', 'tar', 'porque', 'porquê', 'querer', 'entender', 'bom',
                  'coisa', 'cara', 'dizer', 'gostar', 'nada', 'pro', 'ngm', 'vou', 'mto', 'to', 'deixa','começou', 'abrir']

def limpa_texto(texto, stopwords=stopwords):
  texto = texto.lower()
  texto = re.sub(r'[0-9]|#[a-z]{1,}|@[a-z]{1,}|\.|\?|,|\!', '', texto)
  texto = ' '.join([word for word in texto.split() if word not in stopwords])
  return texto

In [ ]:
tweets['text_clean'] = tweets['text'].apply(lambda x: limpa_texto(x, stopwords))

Vamos ver como a base está:

In [ ]:
tweets

,text,label,label_nome,text_clean
0,"Que gafe do VídeoShow, colocar uma cena de mor...",0,Negativo,gafe vídeoshow colocar cena morte dia morte ne...
1,Tapoha q prova difícil #MasterChefBR,1,Neutro,tapoha prova difícil
2,A voz da Vanessa da mata é uma coisa maravilho...,2,Positivo,voz vanessa mata maravilhosa
3,poooooooorra Mirian deixa o cara abrir a panel...,0,Negativo,poooooooorra mirian panela
4,#MasterChefBR preciso parar de assistir esse p...,1,Neutro,preciso parar assistir programa vontade cursar...
...,...,...,...,...
865,Gente eu vi uma notícia falando que a Ana Luiz...,1,Neutro,gente vi notícia falando ana luiza mentiu trab...
866,@user Rola dela ir em um talk show? Prefiro o ...,2,Positivo,rola talk show prefiro pode
867,#MasterChefBR no H&amp;H @MChefFabrizio Como ...,0,Negativo,"h&amp;h falou ""tá raivinha coração"" mudou humo..."
868,#masterchefbr hoje vai ter só proteína ein gente,1,Neutro,vai proteína ein gente


Separando em treino e teste

In [ ]:
treino, teste = train_test_split(tweets, test_size=0.3, stratify=tweets['label'], shuffle=True)

Criando um pipeline de modelo que transforma as palavras em números e aplica o algoritmo Naive-Bayes.

In [ ]:
modelo = Pipeline([
    ('vec', CountVectorizer(min_df=1, ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(use_idf=False)),
    ('nb', MultinomialNB())
    ])

modelo.fit(treino['text_clean'], treino['label_nome'])
pred = modelo.predict(teste['text_clean'])
print(classification_report(teste['label_nome'], pred))

              precision    recall  f1-score   support

    Negativo       0.55      0.63      0.58       271
      Neutro       0.53      0.47      0.50       271
    Positivo       0.63      0.60      0.61       271

    accuracy                           0.57       813
   macro avg       0.57      0.57      0.57       813
weighted avg       0.57      0.57      0.57       813



In [ ]:
modelo = Pipeline([
    ('vec', CountVectorizer(min_df=1, ngram_range=(1, 3))),
    ('tfidf', TfidfTransformer(use_idf=False)),
    ('classifier', (KNeighborsClassifier(n_neighbors=5)))

    ])

modelo.fit(treino['text_clean'], treino['label_nome'])
pred = modelo.predict(teste['text_clean'])
print(classification_report(teste['label_nome'], pred))

              precision    recall  f1-score   support

    Negativo       0.71      0.02      0.04       271
      Neutro       0.35      0.92      0.51       271
    Positivo       0.68      0.23      0.34       271

    accuracy                           0.39       813
   macro avg       0.58      0.39      0.29       813
weighted avg       0.58      0.39      0.29       813



In [ ]:
modelo = Pipeline([
    ('vec', CountVectorizer(min_df=1, ngram_range=(1, 3))),
    ('tfidf', TfidfTransformer(use_idf=False)),
    ('classifier', (SVC()))

    ])

modelo.fit(treino['text_clean'], treino['label_nome'])
pred = modelo.predict(teste['text_clean'])
print(classification_report(teste['label_nome'], pred))

              precision    recall  f1-score   support

    Negativo       0.57      0.51      0.54       271
      Neutro       0.47      0.69      0.56       271
    Positivo       0.75      0.46      0.57       271

    accuracy                           0.56       813
   macro avg       0.60      0.56      0.56       813
weighted avg       0.60      0.56      0.56       813



In [ ]:
modelo = Pipeline([
    ('vec', CountVectorizer(min_df=1, ngram_range=(1, 3))),
    ('tfidf', TfidfTransformer(use_idf=False)),
    ('classifier', (RandomForestClassifier(max_depth=100)))

    ])

modelo.fit(treino['text_clean'], treino['label_nome'])
pred = modelo.predict(teste['text_clean'])
print(classification_report(teste['label_nome'], pred))

              precision    recall  f1-score   support

    Negativo       0.59      0.42      0.49       271
      Neutro       0.43      0.68      0.53       271
    Positivo       0.67      0.48      0.56       271

    accuracy                           0.53       813
   macro avg       0.57      0.53      0.53       813
weighted avg       0.57      0.53      0.53       813



In [ ]:
modelo = Pipeline([
    ('vec', CountVectorizer(min_df=1, ngram_range=(1, 3))),
    ('tfidf', TfidfTransformer(use_idf=False)),
    ('classifier', (XGBClassifier(n_estimators=100)))

    ])

modelo.fit(treino['text_clean'], treino['label'])
pred = modelo.predict(teste['text_clean'])
print(classification_report(teste['label'], pred))

              precision    recall  f1-score   support

           0       0.56      0.37      0.45       271
           1       0.42      0.69      0.52       271
           2       0.63      0.45      0.52       271

    accuracy                           0.50       813
   macro avg       0.54      0.50      0.50       813
weighted avg       0.54      0.50      0.50       813



## Redes Neurais

In [ ]:
modelo = Pipeline([
    ('vec', CountVectorizer(min_df=1, ngram_range=(1, 3))),
    ('tfidf', TfidfTransformer(use_idf=False)),
    ('classifier', (MLPClassifier(hidden_layer_sizes=(100, 100, 100, 100), verbose=True, tol=0.000001)))
    ])

modelo.fit(treino['text_clean'], treino['label'])
pred = modelo.predict(teste['text_clean'])
print(classification_report(teste['label'], pred))

Iteration 1, loss = 1.10144385
Iteration 2, loss = 1.07655703
Iteration 3, loss = 0.99651094
Iteration 4, loss = 0.76523507
Iteration 5, loss = 0.35980945
Iteration 6, loss = 0.07014100
Iteration 7, loss = 0.01671983
Iteration 8, loss = 0.00871904
Iteration 9, loss = 0.00695313
Iteration 10, loss = 0.00619673
Iteration 11, loss = 0.00600795
Iteration 12, loss = 0.00616607
Iteration 13, loss = 0.00584924
Iteration 14, loss = 0.00601404
Iteration 15, loss = 0.00637247
Iteration 16, loss = 0.00592021
Iteration 17, loss = 0.00648635
Iteration 18, loss = 0.00690212
Iteration 19, loss = 0.00584727
Iteration 20, loss = 0.00587416
Iteration 21, loss = 0.00616580
Iteration 22, loss = 0.00639091
Iteration 23, loss = 0.00557044
Iteration 24, loss = 0.00637693
Iteration 25, loss = 0.00623237
Iteration 26, loss = 0.00590081
Iteration 27, loss = 0.00725799
Iteration 28, loss = 0.00739411
Iteration 29, loss = 0.00627556
Iteration 30, loss = 0.00620986
Iteration 31, loss = 0.00582477
Iteration 32, los



### Usando um modelo pré-treinado

In [ ]:
!pip install transformers sentencepiece tokenizers xformers

from transformers import pipeline
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path, device='cuda')

In [ ]:
sentiment_task('Eu adoro Ciência de Dados, acho que é um campo muito interessante')

[{'label': 'positive', 'score': 0.9307498335838318}]

In [ ]:
sentiment_task('Bovespa sobe e atinge maior patamar em quase 2 anos')

[{'label': 'positive', 'score': 0.5674303770065308}]

In [ ]:
def sentimento(texto):
  return sentiment_task(texto)[0]['label']

sentiment_task.model.config.id2label = {0: 'Negativo', 1: 'Neutro', 2: 'Positivo'}


In [ ]:
tweets['predicted'] = tweets['text'].progress_apply(sentimento)

  0%|          | 9/2709 [00:00<01:56, 23.21it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 2709/2709 [00:46<00:00, 57.88it/s]


In [ ]:
tweets

,text,label,label_nome,predicted
0,"Que gafe do VídeoShow, colocar uma cena de mor...",0,Negativo,Negativo
1,Tapoha q prova difícil #MasterChefBR,1,Neutro,Negativo
2,A voz da Vanessa da mata é uma coisa maravilho...,2,Positivo,Positivo
3,poooooooorra Mirian deixa o cara abrir a panel...,0,Negativo,Negativo
4,#MasterChefBR preciso parar de assistir esse p...,1,Neutro,Negativo
...,...,...,...,...
865,Gente eu vi uma notícia falando que a Ana Luiz...,1,Neutro,Negativo
866,@user Rola dela ir em um talk show? Prefiro o ...,2,Positivo,Neutro
867,#MasterChefBR no H&amp;H @MChefFabrizio Como ...,0,Negativo,Neutro
868,#masterchefbr hoje vai ter só proteína ein gente,1,Neutro,Neutro


In [ ]:
print(classification_report(tweets['label_nome'], tweets['predicted']))

              precision    recall  f1-score   support

    Negativo       0.79      0.85      0.82       903
      Neutro       0.73      0.73      0.73       903
    Positivo       0.87      0.81      0.84       903

    accuracy                           0.80      2709
   macro avg       0.80      0.80      0.80      2709
weighted avg       0.80      0.80      0.80      2709



In [ ]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli",
                      device='cuda')

In [ ]:
classes = ['Política', 'Economia', 'Esportes']

classifier('O dólar está em queda por conta das sanções da China', classes)

{'sequence': 'O dólar está em queda por conta das sanções da China',
 'labels': ['Economia', 'Política', 'Esportes'],
 'scores': [0.6596757769584656, 0.185186967253685, 0.15513724088668823]}

In [ ]:
classifier('O dólar está em queda por conta das sanções da China', classes)['labels'][0]

'Economia'

In [ ]:
classifier('O presidente americano afirmou que o acordo com o Brasil será bastante positivo para ambos os lados', classes)['labels'][0]

'Política'